<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/oai_client_cost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copyright (c) Microsoft Corporation. All rights reserved. 

Licensed under the MIT License.

# Use AutoGen's OpenAIWrapper for cost estiomation


## Requirements

AutoGen requires `Python>=3.8`:
```bash
pip install "pyautogen"
```

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.


In [2]:
import autogen

# config_list = autogen.config_list_from_json(
#     "OAI_CONFIG_LIST",
#     filter_dict={
#         "model": ["gpt-3.5-turbo", "gpt-4-1106-preview"],
#     },
# )

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-35-turbo-1106"],
    },
)



It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well). Only the gpt-4 models are kept in the list based on the filter condition.

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-4',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-4-32k',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]
```

You can set the value of config_list in any way you prefer. Please refer to this [notebook](https://github.com/microsoft/autogen/blob/main/notebook/oai_openai_utils.ipynb) for full code examples of the different methods.

## OpenAIWrapper with cost estimation

In [6]:
from autogen import OpenAIWrapper

client = OpenAIWrapper(config_list=config_list)
messages = [{'role': 'user', 'content': 'Can you give me 3 useful tips on learning Python? Keep it simple and short.'},]
response = client.create(messages=messages, model="gpt-35-turbo-1106", cache_seed=None)
print(response.cost)

0.0001515


## Usage Summary

When creating a instance of OpenAIWrapper, cost of all completions from the same instance is recorded. You can call `print_usage_summary()` to checkout your usage summary. To clear up, use `clear_usage_summary()`.


In [3]:
from autogen import OpenAIWrapper

client = OpenAIWrapper(config_list=config_list)
messages = [{'role': 'user', 'content': 'Can you give me 3 useful tips on learning Python? Keep it simple and short.'},]
client.print_usage_summary() # print usage summary

No usage summary. Please call "create" first.


In [14]:
# The first creation
# By default, cache_seed is set to 41 and enabled. If you don't want to use cache, set cache_seed to None.
response = client.create(messages=messages, model="gpt-35-turbo-1106", cache_seed=52)
client.print_usage_summary(mode='both') # default mode is 'both', can be set to 'actual' or 'all' to only print actual usage or all usage
client.print_usage_summary(mode='actual') # print actual usage summary
client.print_usage_summary(mode='all') # print all usage summary

----------------------------------------------------------------------------------------------------
Usage summary excluding cached usage: 
Total cost: 0.00015
* Model 'gpt-35-turbo': cost: 0.00015, prompt_tokens: 25, completion_tokens: 58, total_tokens: 83

Usage summary including cached usage: 
Total cost: 0.00027
* Model 'gpt-35-turbo': cost: 0.00027, prompt_tokens: 50, completion_tokens: 100, total_tokens: 150
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Usage summary excluding cached usage: 
Total cost: 0.00015
* Model 'gpt-35-turbo': cost: 0.00015, prompt_tokens: 25, completion_tokens: 58, total_tokens: 83
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Usage summary includi

In [16]:
# take out cost
print(client.actual_usage_summary)
print(client.all_usage_summary)

{'total_cost': 0.0001535, 'gpt-35-turbo': {'cost': 0.0001535, 'prompt_tokens': 25, 'completion_tokens': 58, 'total_tokens': 83}}
{'total_cost': 0.00027499999999999996, 'gpt-35-turbo': {'cost': 0.00027499999999999996, 'prompt_tokens': 50, 'completion_tokens': 100, 'total_tokens': 150}}


In [17]:
# Since cache is enabled, the same completion will be returned from cache, which will not incur any actual cost. 
# So acutal cost incurred is still 0.00012, but total cost including cache usage is 0.00024.
response = client.create(messages=messages, model="gpt-35-turbo-1106", cache_seed=51)
client.print_usage_summary()

----------------------------------------------------------------------------------------------------
Usage summary excluding cached usage: 
Total cost: 0.00015
* Model 'gpt-35-turbo': cost: 0.00015, prompt_tokens: 25, completion_tokens: 58, total_tokens: 83

Usage summary including cached usage: 
Total cost: 0.0004
* Model 'gpt-35-turbo': cost: 0.0004, prompt_tokens: 75, completion_tokens: 142, total_tokens: 217
----------------------------------------------------------------------------------------------------


In [18]:
# Usage summary is cleared.
client.clear_usage_summary() # clear usage summary
client.print_usage_summary()

No usage summary. Please call "create" first.


In [19]:
# all completions are returned from cache, so no actual cost incurred.
response = client.create(messages=messages, model="gpt-35-turbo-1106", cache_seed=51)
client.print_usage_summary()

----------------------------------------------------------------------------------------------------
No actual cost incurred (all completions are using cache).

Usage summary including cached usage: 
Total cost: 0.00012
* Model 'gpt-35-turbo': cost: 0.00012, prompt_tokens: 25, completion_tokens: 42, total_tokens: 67
----------------------------------------------------------------------------------------------------
